In [1]:
from IPython.display import Markdown, display

## Links to Project Resources

- [Trello board](https://trello.com/invite/b/BWnRAtKJ/3e7ce03017000289323e762d0ed2e304/histaware)
- [Notion Wiki](https://www.notion.so/HistAware-529aba41f84946b19d493394ef6a2748)

# Part I: Text selection

In this first phase of the project, we approach the first problem of selecting texts similar texts. Intially the scope of the research is focused on texts that deal with `energy`. However, this scope might change and/or might be expanded.

**Phases of Part I:**
- **Validate the approach to the project**:
    1. Decide whether to use title and paragraphs or only one of the two
    2. Find the most efficient way to read all the xml files
    3. Begin to label a golden set of texts that are within the scope of the research AND select the most important keywords that will be used to search for similar texts
    4. Run the text similarity ML algorithm
    5. Have the teaching assistant go throught the selection and identify mistakes
- **To think about**: how to keep the relevant information about the text fragment (i.e. newspaper origin and date)?
- **Decide the tools to use for text selection**. Current choices are:
    - Use `sentence-transformers` from UKPLab (https://github.com/UKPLab/sentence-transformers)
        - Generate embeddings on sentences (max 512 words)
        - Find similar texts
    - Use `faiss` from Facebook AI (https://github.com/facebookresearch/faiss)
        - Less documentation but seemingly more scalable
    - Use ASReview from Utrecht University ()
        - A meeting with Jonathan or Raul is necessary to understand the feasibility of this approach

### Import statements

In [31]:
import numpy as np
import pandas as pd
import logging
import re
from datetime import datetime
import xml.etree.ElementTree as et 
import collections
import sys
import os
import gzip
import shutil
import xml.etree.ElementTree as ET

%matplotlib inline
%config InlineBackend.figure_format='retina'


#### Just some code to print debug information to stdout
np.set_printoptions(threshold=100)

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)
#### /print debug information to stdout

# Find path of data folder
main_path = sys.path
# To go back to main folder
sys.path.insert(0, "..")

## Delpher Dataset

### Create a catalogue of the files

#### Find the location of each article

We save the file path and the file name into a dictionary. Then we transform the dictionary into a DataFrame so that we can later keep track of the index at which the parsing got stopped/interrupted (Dictionaries in Python do not have an order)

In [3]:
def iterate_directory(path_dir, file_type):
    """Iterate over the `path_dir` and its children and
    create a dictionary of
        - name
        - path
        - dir
    names of files found
    """
    rootdir = main_path[0]+path_dir
    file_names = {}
    list_names = []

    for subdir, dirs, files in os.walk(rootdir, topdown=True):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(str(file_type)):
                file_names["article_name"] = file
                file_names["article_path"] = filepath
                file_names["article_dir"] = subdir
                list_names.append(file_names)
                file_names = {}

    return(list_names)

In [4]:
xml_article_names = iterate_directory("/data/1950/",".xml")
article_names = pd.DataFrame.from_dict(xml_article_names)

#### Find location of each metadata and "ungizp" them

**ATTENTION**: This loop writes to memory the content of the .gz files!
It is long and "resource intensive"

In [6]:
def iterate_directory_gz(path_dir,file_type):
    """Iterate over the `path_dir` and its children and
    create a dictionary of
        - name
        - path
        - dir
        - content
    of .gz files found.
    """
    rootdir = main_path[0]+path_dir
    gz_content = {}
    list_gzs = []
    
    for subdir, dirs, files in os.walk(rootdir, topdown=True):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(str(file_type)):
                # Create list of dict
                with gzip.open(filepath, 'rb') as f, \
                    open(filepath+".xml", "wb") as r:
                    # This changes the name to a name without backslashes
                    # I.e. from "/" to ":"
                    gz_content["metadata_name"] = filepath+".xml"
                    gz_content["metadata_dir"] = subdir
                    gz_content["metadata_path"] = filepath+".xml"
                    # Ungzipping and writing to .xml
                    shutil.copyfileobj(f, r, 65536)
                    
                    list_gzs.append(gz_content)
                    gz_content = {}
    
    return(list_gzs)

In [7]:
gz_metadata_files = iterate_directory_gz("/data/1950/",".gz")
metadata_files = pd.DataFrame.from_dict(gz_metadata_files)

### Parse XML

#### Parse articles

In [8]:
def parse_XML_article(path, art_dir, title, index):
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    Takes the filepath, file title and index integer of the df
    """
    
    xtree = et.parse(path)
    xroot = xtree.getroot()
    article = {}
    dict_articles = {}
    
    # Parse the date with regex
    match = re.search(r'\d{4}[/]\d{2}[-]\d{2}', path)
    date = datetime.strptime(match.group(), '%Y/%m-%d').date()
    
    for i, node in enumerate(xroot):
        article["article_name"] = str(title)
        article["date"] = str(date)
        article["index"] = index
        article["filepath"] = path
        article["dir"] = art_dir
        if node.tag != "p":
            article[node.tag] = node.text
        else:
            article[node.tag+"_"+str(i)] = node.text
        dict_articles[index] = article

    # Returns dict of dicts to speed up the parsing
    return dict_articles

#### Parse metadata

In [11]:
metadata_files["metadata_name"][1]

'../data/1950/04-27/DDD_110584873/DDD:ddd:110584873:mpeg21.didl.xml.gz.xml'

In [39]:
path = metadata_files["metadata_path"][1]
with open(metadata_files["metadata_path"][1], "r") as f:
        a = f.read()
        print(a)
        b = ET.parse(a)
        #root = tree.getroot(a)
        print(b)
        #a = print(unknown)
#xtree = parseString(path)
#print(xtree)

<didl:DIDL xmlns:didl="urn:mpeg:mpeg21:2002:02-DIDL-NS" xmlns:ddd="http://www.kb.nl/namespaces/ddd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dcmitype="http://purl.org/dc/dcmitype/" xmlns="urn:mpeg:mpeg21:2002:02-DIDL-NS" xmlns:dcx="http://krait.kb.nl/coop/tel/handbook/telterms.html" xmlns:didmodel="urn:mpeg:mpeg21:2002:02-DIDMODEL-NS" xmlns:srw_dc="info:srw/schema/1/dc-v1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
				<didl:Item dc:identifier="ddd:110584873:mpeg21">
					<didl:Component dc:identifier="ddd:110584873:mpeg21:metadata">
						<didl:Descriptor>
							<didl:Statement mimeType="text/plain" dc:type="role">metadata</didl:Statement>
						</didl:Descriptor>
						<didl:Resource mimeType="text/xml">
							<srw_dc:dcx>
								<dc:title>De Telegraaf</dc:title>
								<dc:identifier xsi:type="dcx:PPN">832675288</dc:identifier>
								<dc:date>1950-04-27</dc:date>
								<dcterms:temporal>Dag</dcterms:temporal>
	

OSError: [Errno 63] File name too long: '<didl:DIDL xmlns:didl="urn:mpeg:mpeg21:2002:02-DIDL-NS" xmlns:ddd="http://www.kb.nl/namespaces/ddd" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:dcmitype="http://purl.org/dc/dcmitype/" xmlns="urn:mpeg:mpeg21:2002:02-DIDL-NS" xmlns:dcx="http://krait.kb.nl/coop/tel/handbook/telterms.html" xmlns:didmodel="urn:mpeg:mpeg21:2002:02-DIDMODEL-NS" xmlns:srw_dc="info:srw/schema/1/dc-v1.1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n\t\t\t\t<didl:Item dc:identifier="ddd:110584873:mpeg21">\n\t\t\t\t\t<didl:Component dc:identifier="ddd:110584873:mpeg21:metadata">\n\t\t\t\t\t\t<didl:Descriptor>\n\t\t\t\t\t\t\t<didl:Statement mimeType="text/plain" dc:type="role">metadata</didl:Statement>\n\t\t\t\t\t\t</didl:Descriptor>\n\t\t\t\t\t\t<didl:Resource mimeType="text/xml">\n\t\t\t\t\t\t\t<srw_dc:dcx>\n\t\t\t\t\t\t\t\t<dc:title>De Telegraaf</dc:title>\n\t\t\t\t\t\t\t\t<dc:identifier xsi:type="dcx:PPN">8326752

In [ ]:
def parse_and_get_ns(file):
    root = None
    ns = {}
    for elem in ET.iterparse(file):
        if event == "start-ns":
            if elem[0] in ns and ns[elem[0]] != elem[1]:
                # NOTE: It is perfectly valid to have the same prefix refer
                #     to different URI namespaces in different parts of the
                #     document. This exception serves as a reminder that this
                #     solution is not robust.    Use at your own peril.
                raise KeyError("Duplicate prefix with different URI found.")
            ns[elem[0]] = "{%s}" % elem[1]
        elif event == "start":
            if root is None:
                root = elem
    return ET.ElementTree(root), ns

In [32]:
def parse_and_get_ns(file):
    events = "start", "start-ns"
    root = None
    ns = {}
    for event, elem in ET.iterparse(file, events):
        if event == "start-ns":
            if elem[0] in ns and ns[elem[0]] != elem[1]:
                # NOTE: It is perfectly valid to have the same prefix refer
                #     to different URI namespaces in different parts of the
                #     document. This exception serves as a reminder that this
                #     solution is not robust.    Use at your own peril.
                raise KeyError("Duplicate prefix with different URI found.")
            ns[elem[0]] = "{%s}" % elem[1]
        elif event == "start":
            if root is None:
                root = elem
    return ET.ElementTree(root), ns

In [35]:
tree, ns = parse_and_get_ns(metadata_files["metadata_name"][1])
ns
#{u'content': '{http://purl.org/rss/1.0/modules/content/}',
#u'dc': '{http://purl.org/dc/elements/1.1/}'}
item = tree.find("/dcx")
item.findtext(ns['dc']+"creator")
#'Joe McGroin'

None


/Users/leonardovida/dev/HistAware/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './dcx'
  """


In [19]:
def parse_XML_metadata(path, met_dir, title, index):
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    
    Takes the filepath, file title and index integer of the df
    """
    
    xtree = et.parse(path)
    xroot = xtree.getroot()
    metadata = {}
    dict_metadata = {}
    
    # Parse the date with regex
    match = re.search(r'\d{4}[/]\d{2}[-]\d{2}', path)
    date = datetime.strptime(match.group(), '%Y/%m-%d').date()
    
    for i, node in enumerate(xroot):
        metadata["article_name"] = str(title)
        metadata["date"] = str(date)
        metadata["index"] = index
        metadata["filepath"] = path
        metadata["dir"] = met_dir
        if node.tag = "p":
            metadata[node.tag] = node.text
        else:
            metadata[node.tag+"_"+str(i)] = node.text
        dict_metadata[index] = metadata

    return dict_metadata

**Utils Addendum**

To search for an `article_path` or `article_name` given the other, use the following:

In [301]:
#a = df_file_names.loc[df_file_names['article_name'] == "DDD_110637387_0004_articletext.xml"]
#a = df_file_names.iloc[0]
c = df_file_names.iloc[500000]

### Iterate through the files given

Currently, this loop takes ~0.012s for each parsing. This is extremely slow and it's not due to the `parse_XML` function (which is efficient), but instead it's because of the `concat` between series. 

In this way 100.000 documents take around 20 minutes to be parsed.
- If possible, substitute the concat statement with something more efficient!

In [8]:
from itertools import chain

def iterate_files(files):
    """Iterate through files `files`, parse them and concatenate
    the result to be saved as a DataFrame in a feather object (.ftr)
    """
    main = None
    previous_i = 0
    current_i = 0
    i = 0
    n = 0
    cnt = 0
    dict_articles = {}
    
    for index, row in files.iterrows():
        dict_articles.update(
            parse_XML_metadata(
                path = row["article_path"],
                art_dir = row["article_dir"], 
                title = row["article_name"],
                index = index))
        # Each X, save the file in a .ftr
        if (i == 2000):
            current_i = current_i + i
            file_path = path[0]+"/data/processed/processed_data_"+str(previous_i)+"_"+str(current_i)+".ftr"
            main = pd.DataFrame.from_dict(dict_articles).T
            # Stops here for now!
            return(main)
        ############################
            main.to_feather(file_path)
            list_series = []
            main = None
            previous_i = current_i
            i = 0
        # Each 1000 files, print the progress
        if (i % 2000 == 0):
            print("Files parsed: "+str(2000*cnt))
            print("Current file: "+row["article_name"]+"\n")
            cnt += 1
        i += 1

In [ ]:
from itertools import chain

def iterate_metadata(files):
    """Iterate through files `files`, parse them and concatenate
    the result to be saved as a DataFrame in a feather object (.ftr)
    """
    main = None
    previous_i = 0
    current_i = 0
    i = 0
    n = 0
    cnt = 0
    dict_articles = {}
    
    for index, row in files.iterrows():
        dict_articles.update(
            parse_XML_metadata(
                path = row["filepath"],
#                art_dir = row["article_dir"], 
                title = row["filename"],
                content = row["content"],
                index = index))
        # Each X, save the file in a .ftr
        if (i == 2000):
            current_i = current_i + i
            file_path = path[0]+"/data/processed/processed_data_"+str(previous_i)+"_"+str(current_i)+".ftr"
            main = pd.DataFrame.from_dict(dict_articles).T
            # Stops here for now!
            return(main)
        ############################
            main.to_feather(file_path)
            list_series = []
            main = None
            previous_i = current_i
            i = 0
        # Each 1000 files, print the progress
        if (i % 2000 == 0):
            print("Files parsed: "+str(2000*cnt))
            print("Current file: "+row["article_name"]+"\n")
            cnt += 1
        i += 1

In [50]:
a = iterate_files(article_names)

Files parsed: 0
Current file: DDD_011210828_0052_articletext.xml



In [26]:
b = iterate_files(metadata_files)

KeyError: 'article_path'

In [ ]:
b

## Text selection model

### Ingest parsed files

Once we parse all the files present in the example `data-1950` folder, we produce 65 files containing the parsed original data into a format which is more easily readable by a machine. The total weight of the files is 65*10=650MB which is a 5x reduction from the original size of the dataset.

In [7]:
# https://www.sbert.net/docs/
from sentence_transformers import SentenceTransformer, LoggingHandler, util

# These are the pure transformers from huggingface
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

# For saving
import pickle
import csv

# Set searborn settings
rcParams['figure.figsize'] = 12, 8

# Set fixed random seed
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Find GPU on device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

2020-09-04 10:17:04 - PyTorch version 1.6.0 available.


#### Retrieve all the names of the ftr files saved

In [8]:
ftr_file_names = iterate_directory("/data/processed/",".ftr")
ftr_file_names = pd.DataFrame.from_dict(ftr_file_names.items())
ftr_file_names.rename({0: 'ftr_name', 1: 'ftr_path'}, axis=1, inplace=True)

#### Retrieve all the content of the files into a list format

In [9]:
list_ftr = []

# Only gets the first file!!!
for index, row in ftr_file_names.iterrows():
    if index > 1:
        break
    else:
        ftr = pd.read_feather(row["ftr_path"])
        list_ftr.append(ftr)

In [11]:
ftr

,article_name,date,index,title,p_1,p_2,p_3,p_4,p_5,p_6,...,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36
0,DDD_010850957_0035_articletext.xml,1950-03-08,10001,Onderdistrict West zet verkiezingscampagne in,3 bruggen het eerste resultaat Op een j.I. Din...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,DDD_010850957_0040_articletext.xml,1950-03-08,10002,DINGEN VAN DE DAG,t/„.t„;j.'_„ Gisteren heeft Verleiding... Dagw...,Onze jonge vriend Van Voorst moest Zondag j.l....,"Vanaf deze plaats wil de jonge man, Van Voorst...",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,DDD_010850957_0007_articletext.xml,1950-03-08,10003,None,"..__, , — 1 ._..... , , .... Hedenavond BELLEV...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,DDD_010850957_0023_articletext.xml,1950-03-08,10004,Zaandam-Amsterdam niet meer per watertaxi,"Op 17 Maart a.s. worden de watertaxi's, die de...",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,DDD_010850957_0064_articletext.xml,1950-03-08,10005,DUW-arbeiders protesteren tegen ronselpraktijk...,Gistermorgen heeft een onderhoud plaatsgevonde...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,DDD_011155304_0004_articletext.xml,1950-03-06,19996,Ambtenarij en belastingen,De Franse regeringen zijn uitgegaan van de gro...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9996,DDD_011155304_0131_articletext.xml,1950-03-06,19997,Hockey (heren),le klasse Amsterdam—H O C ... I—l Be Fair—Hilv...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9997,DDD_011155304_0043_articletext.xml,1950-03-06,19998,"Beter kleden, roken en meer kwaliteit",Uit andere cijfers blijkt evenwel onomstotelij...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9998,DDD_011155304_0128_articletext.xml,1950-03-06,19999,Korfbal,Blauw Wit—Archipel 7—3 Koog Zaandijk—S V K I—6...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


#### Create a list with only the text (paragraphs) and not the other variables

In [10]:
list_sentences = []

for index, row in ftr.iterrows():
    for i in range(1,ftr.shape[1]-4):
        p = "p_"+str(i)
        if row[p] and row[p] is not None:
            list_sentences.append(row[p])

In [50]:
test = pd.DataFrame(list_sentences).reset_index()
test.rename({0: 'text'}, axis=1, inplace=True)

In [55]:
test.to_csv(path[0]+"/data"+"/list_sentences.csv",
            sep=",",
            quotechar='"',
            header=["index","text"],
            index=False)

#with open(path[0]+"/data"+"/list_sentences.csv", 'w') as myfile:
#    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#    wr.writerow(list_sentences)

#with open('list_sentences.pkl', "wb") as fOut:
#    pickle.dump(list_sentences, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
# Temp list
sen = list_sentences

### Use the multilingual model pre-trained on 10+ languages

### Play around with `SBERT`

The model is the `distiluse-base-multilingual-cased` model. From (sbert)[https://www.sbert.net/docs/pretrained_models.html]

In [23]:
# Create embeddings
model = SentenceTransformer('distiluse-base-multilingual-cased', device=device)

# Load paragraphs
sentences = sen

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

2020-09-01 14:57:56 - Load pretrained SentenceTransformer: distiluse-base-multilingual-cased
2020-09-01 14:57:56 - Did not find a '/' or '\' in the name. Assume to download model from server.
2020-09-01 14:57:56 - Load SentenceTransformer from folder: /Users/leonardovida/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_distiluse-base-multilingual-cased.zip
2020-09-01 14:57:56 - loading configuration file /Users/leonardovida/.cache/torch/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_distiluse-base-multilingual-cased.zip/0_DistilBERT/config.json
2020-09-01 14:57:56 - Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_hidden_states": true,
  "output_past": true,



Sentence: t/„.t„;j.'_„ Gisteren heeft Verleiding... Dagwacnt op één van zyn tochten door de stad alle mogelyke moeite gedaan de parkjes te omzeilen. De verleiding was groot om neer te zinken op één van de banken en te genieten van de zalige zomers aandoende zonneschyn. Laat hy u vertellen, dat hij niet bezweken is, maar manmoedig doorstapte naar het uitgestippelde doel. Maar zomer was het. De kinderen, die met meester of juffrouw wandelden, maakten extra hard lawaai, stampten met de schoenen, zongen en sprongen. De kinderwagens met de jonge spruiten erin stonden in de stadsparken wiel aan wiel. De jonge moeders keuvelden en breiden. Opmerkeiyk hoe goed dat samen gaat. Maar laten we nu stoppen. We hebben vandaag nog meer te doen. Het doel van gistermiddag was een gouden bruiloft, die op komst is. Gouden feest ™™£* straat 149 ni woont het echtpaar K. F. de Dood—Christina Johanna Elders. Vandaag, Woensdag 8 Maart, gaan de bruidsdagen in. En de heuglijke dag, waarop zy vflftig jaar gelede

In [34]:
# Sentence mining from sentence-transformers
sentences = sen[1:40]
paraphrases = util.paraphrase_mining(
    model,
    sentences,
    corpus_chunk_size=20, #len(sentences)
    query_chunk_size=20,
    top_k=20,
    max_pairs=5)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

Maar het waren juist deze geheime gegevens, die Louwers in handen wilde krijgen. Want de Tsjechoslowaakse industrie heeft op dit gebied een reusachtige voorsprong. Of nemen wij Philips! Toen de Duitsers ons land binnenvielen, vertrokken uit Eindhoven 30 auto's, volgeladen met geheime papieren: de productiegeheimen van Philips! Die moesten in veiligheid. De rest was niet belangrijk. Geen mens buiten de vesting van Philips komt over deze geheimen iets te weten. En binnen de vesting is een wetenschappelijk systeem uitgewerkt, dat er voor zorgt, dat de lagere ingenieurs en technici slechts gedeelten van een bepaald productieproces leren kennen, waarmee zij niets kunnen beginnen. Verteld wordt, dat eens een delegatie van Japanse belangstellenden Philips kwam bezoeken. De heren werden „overal" rondgeleid, ze kregen een pracht van een diner aangeboden. Maar de Philipsdirectie vermoedde dat sommigen van hen micro-fototoestellen by zich hadden. Fouilleren kon men de „geëerde gasten" niet. Dus w

In [ ]:
#with open('paraphrase_test.pkl', "wb") as fOut:
#    pickle.dump(paraphrase, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
# Save embeddings
import pickle

with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': sentences, 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

    
    
#Load sentences & embeddings from disc
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    stored_embeddings = stored_data['embeddings']

Maar het waren juist deze geheime gegevens, die Louwers in handen wilde krijgen. Want de Tsjechoslowaakse industrie heeft op dit gebied een reusachtige voorsprong. Of nemen wij Philips! Toen de Duitsers ons land binnenvielen, vertrokken uit Eindhoven 30 auto's, volgeladen met geheime papieren: de productiegeheimen van Philips! Die moesten in veiligheid. De rest was niet belangrijk. Geen mens buiten de vesting van Philips komt over deze geheimen iets te weten. En binnen de vesting is een wetenschappelijk systeem uitgewerkt, dat er voor zorgt, dat de lagere ingenieurs en technici slechts gedeelten van een bepaald productieproces leren kennen, waarmee zij niets kunnen beginnen. Verteld wordt, dat eens een delegatie van Japanse belangstellenden Philips kwam bezoeken. De heren werden „overal" rondgeleid, ze kregen een pracht van een diner aangeboden. Maar de Philipsdirectie vermoedde dat sommigen van hen micro-fototoestellen by zich hadden. Fouilleren kon men de „geëerde gasten" niet. Dus w

### Playing around with BERTje

In [18]:
from transformers import BertTokenizer, BertModel


tokenizer = BertTokenizer.from_pretrained("wietsedv/bert-base-dutch-cased")
model = BertModel.from_pretrained("wietsedv/bert-base-dutch-cased")

2020-09-02 11:23:32 - Lock 5397104528 acquired on /Users/leonardovida/.cache/torch/transformers/75d9be4cc7910048b3bdd477c435ffc46330193705f74eaf9a4f375cd3be28b2.1e00a56207196ed1759c49bdd1fa93c2fb20273d59fabb0c4c8092f7beb773c2.lock
2020-09-02 11:23:32 - https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/vocab.txt not found in cache or force_download set to True, downloading to /Users/leonardovida/.cache/torch/transformers/tmppdbo_bik


2020-09-02 11:23:33 - storing https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/vocab.txt in cache at /Users/leonardovida/.cache/torch/transformers/75d9be4cc7910048b3bdd477c435ffc46330193705f74eaf9a4f375cd3be28b2.1e00a56207196ed1759c49bdd1fa93c2fb20273d59fabb0c4c8092f7beb773c2
2020-09-02 11:23:33 - creating metadata file for /Users/leonardovida/.cache/torch/transformers/75d9be4cc7910048b3bdd477c435ffc46330193705f74eaf9a4f375cd3be28b2.1e00a56207196ed1759c49bdd1fa93c2fb20273d59fabb0c4c8092f7beb773c2
2020-09-02 11:23:33 - Lock 5397104528 released on /Users/leonardovida/.cache/torch/transformers/75d9be4cc7910048b3bdd477c435ffc46330193705f74eaf9a4f375cd3be28b2.1e00a56207196ed1759c49bdd1fa93c2fb20273d59fabb0c4c8092f7beb773c2.lock
2020-09-02 11:23:33 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/vocab.txt from cache at /Users/leonardovida/.cache/torch/transformers/75d9be4cc7910048b3bdd477c435ffc46330193705f74

2020-09-02 11:23:34 - Lock 5357248464 acquired on /Users/leonardovida/.cache/torch/transformers/6702c5c53edb76b65d71f73ff2d9811ba62f16257ea58e36dedceffd71290a6a.1a78bd120fe46d78b55efa59f4ffa1dafcc9242743ab9fd6629d1b56672c9119.lock
2020-09-02 11:23:34 - https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/config.json not found in cache or force_download set to True, downloading to /Users/leonardovida/.cache/torch/transformers/tmpqyn8q3s_


2020-09-02 11:23:34 - storing https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/config.json in cache at /Users/leonardovida/.cache/torch/transformers/6702c5c53edb76b65d71f73ff2d9811ba62f16257ea58e36dedceffd71290a6a.1a78bd120fe46d78b55efa59f4ffa1dafcc9242743ab9fd6629d1b56672c9119
2020-09-02 11:23:34 - creating metadata file for /Users/leonardovida/.cache/torch/transformers/6702c5c53edb76b65d71f73ff2d9811ba62f16257ea58e36dedceffd71290a6a.1a78bd120fe46d78b55efa59f4ffa1dafcc9242743ab9fd6629d1b56672c9119
2020-09-02 11:23:34 - Lock 5357248464 released on /Users/leonardovida/.cache/torch/transformers/6702c5c53edb76b65d71f73ff2d9811ba62f16257ea58e36dedceffd71290a6a.1a78bd120fe46d78b55efa59f4ffa1dafcc9242743ab9fd6629d1b56672c9119.lock
2020-09-02 11:23:34 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/wietsedv/bert-base-dutch-cased/config.json from cache at /Users/leonardovida/.cache/torch/transformers/6702c5c53edb76b65d71f73ff2d

2020-09-02 11:23:35 - Lock 5390180048 acquired on /Users/leonardovida/.cache/torch/transformers/e5754f612ca0f16edba5b775fdddba806751f5e4b87c5e7f16cc0c8d8d17df4d.b7c03627733fd0712f078a4d3a31ad964550f50a6113efdf874ecbcf5ddf6b53.lock
2020-09-02 11:23:35 - https://cdn.huggingface.co/wietsedv/bert-base-dutch-cased/pytorch_model.bin not found in cache or force_download set to True, downloading to /Users/leonardovida/.cache/torch/transformers/tmp1wjgs76_


2020-09-02 11:24:36 - storing https://cdn.huggingface.co/wietsedv/bert-base-dutch-cased/pytorch_model.bin in cache at /Users/leonardovida/.cache/torch/transformers/e5754f612ca0f16edba5b775fdddba806751f5e4b87c5e7f16cc0c8d8d17df4d.b7c03627733fd0712f078a4d3a31ad964550f50a6113efdf874ecbcf5ddf6b53
2020-09-02 11:24:36 - creating metadata file for /Users/leonardovida/.cache/torch/transformers/e5754f612ca0f16edba5b775fdddba806751f5e4b87c5e7f16cc0c8d8d17df4d.b7c03627733fd0712f078a4d3a31ad964550f50a6113efdf874ecbcf5ddf6b53
2020-09-02 11:24:36 - Lock 5390180048 released on /Users/leonardovida/.cache/torch/transformers/e5754f612ca0f16edba5b775fdddba806751f5e4b87c5e7f16cc0c8d8d17df4d.b7c03627733fd0712f078a4d3a31ad964550f50a6113efdf874ecbcf5ddf6b53.lock
2020-09-02 11:24:36 - loading weights file https://cdn.huggingface.co/wietsedv/bert-base-dutch-cased/pytorch_model.bin from cache at /Users/leonardovida/.cache/torch/transformers/e5754f612ca0f16edba5b775fdddba806751f5e4b87c5e7f16cc0c8d8d17df4d.b7c036

2020-09-02 11:24:40 - All model checkpoint weights were used when initializing BertModel.

2020-09-02 11:24:40 - All the weights of BertModel were initialized from the model checkpoint at wietsedv/bert-base-dutch-cased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use BertModel for predictions without further training.
